## Help your hedge fund manager!

You have two datasets at your disposal: `Balance_Sheet.xlsx` and `Income_Statement.xlsx`. Both these datasets have three columns in common:
- `"Company"`: The company's ticker name.
- `"comp_type"` The type of industry the company in question belongs to. It is either `"tech"` for companies in the technology industry, `"fmcg"` for companies in the fast-moving consumer goods industry, and `"real_est"` for companies in the real estate industry. 
- `"Year"`: The year the company's information is from.

The rest of the columns in the datasets contain information from the financial statement of the `"Company"` in question. Note that the columns in `Balance_Sheet.xlsx` only contain financial information from the balance sheet. Similarly, the columns in `Income_Statement.xlsx` only contain financial information from the income statement. The columns are named accordingly. For instance, the column `"Total Liab"` from `Balance_Sheet.xlsx` is the total liability.

In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
from difflib import get_close_matches

# Load the datasets
balance_sheet = pd.read_excel('Balance_Sheet.xlsx')
income_statement = pd.read_excel('Income_Statement.xlsx')

# Standardize column names to ensure merge works
balance_sheet.columns = balance_sheet.columns.str.strip().str.lower()
income_statement.columns = income_statement.columns.str.strip().str.lower()

# Merge keys
merge_keys = ['company', 'comp_type', 'year']

# Merge the datasets
df = pd.merge(balance_sheet, income_statement, on=merge_keys)

# Create df_ratios DataFrame
df_ratios = df[merge_keys].copy()

# Leverage Ratio = Total Liabilities / Total Stockholder Equity
df_ratios['leverage_ratio'] = df['total liab'] / df['total stockholder equity']

# Profitability Ratio = Operating Income / Total Revenue
df_ratios['profitability_ratio'] = df['operating income'] / df['total revenue']

# Handle any infinite or NaN values
df_ratios = df_ratios.replace([np.inf, -np.inf], np.nan)
df_ratios = df_ratios.dropna()

# Task 1: Which company type has the lowest profitability ratio?
avg_profitability_by_type = df_ratios.groupby('comp_type')['profitability_ratio'].mean()
lowest_profitability = avg_profitability_by_type.idxmin()
print(f"Company type with lowest profitability ratio: {lowest_profitability}")

# Task 2: Which company type has the highest leverage ratio?
avg_leverage_by_type = df_ratios.groupby('comp_type')['leverage_ratio'].mean()
highest_leverage = avg_leverage_by_type.idxmax()
print(f"Company type with highest leverage ratio: {highest_leverage}")

# Task 3: Relationship between leverage and profitability for real estate companies
real_estate_data = df_ratios[df_ratios['comp_type'] == 'real_est']
correlation = real_estate_data['leverage_ratio'].corr(real_estate_data['profitability_ratio'])

if correlation > 0.3:
    relationship = "positive"
elif correlation < -0.3:
    relationship = "negative"
else:
    relationship = "no relationship"

print(f"Relationship between leverage and profitability for real estate companies:")
print(f"Correlation coefficient: {correlation:.4f}")
print(f"Relationship: {relationship}")

# Display summary statistics
print("\n=== Summary Statistics ===")
print("\nAverage Profitability by Company Type:")
print(avg_profitability_by_type.sort_values())
print("\nAverage Leverage by Company Type:")
print(avg_leverage_by_type.sort_values())

Company type with lowest profitability ratio: fmcg
Company type with highest leverage ratio: real_est
Relationship between leverage and profitability for real estate companies:
Correlation coefficient: 0.6158
Relationship: positive

=== Summary Statistics ===

Average Profitability by Company Type:
comp_type
fmcg        0.207112
tech        0.273889
real_est    0.299938
Name: profitability_ratio, dtype: float64

Average Leverage by Company Type:
comp_type
tech        1.777448
fmcg        2.997896
real_est    5.692041
Name: leverage_ratio, dtype: float64
